# ARDDoS Data analysis for TFTP, SNMP and SSDP test traffic

In [829]:
# Generic
import os
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import math
import re
import ipaddress
import kaleido

# For showing progress bar of for loops
from progressbar import Bar, ETA, \
    AdaptiveETA, Percentage, \
    ProgressBar 
widgets = [Percentage(),
            ' ', Bar(),
            ' ', ETA(),
            ' ', AdaptiveETA()]
pbar = ProgressBar(widgets=widgets)

# Own functions
from pcapreader import PcapReader
pcapToDf = PcapReader.pcapToDf

# For converting string to ip address in dataframe
from cyberpandas import to_ipaddress 
# For pretty printing dataframes
from tabulate import tabulate 
# For plotting
import plotly.express as px

# Functions

In [830]:
def refineLatexTableStringScientificPaper(latexTableString):
    # Hacks for multiindex dataframe
    multiColFirst = latexTableString.find("\multicolumn")
    latexTableString = latexTableString[:multiColFirst+16] + 'c' + latexTableString[multiColFirst+17:]
    startRemove = latexTableString.find('\multicolumn',multiColFirst+1)
    latexTableString = latexTableString[:startRemove] + latexTableString[startRemove+18:]
    # Convert Level, Amplification factor and packets per sec to multirow:
    latexTableString = latexTableString[:latexTableString.find('Level')] + '\\multirow{2}{*}{Level}' +  latexTableString[latexTableString.find('Level')+len('Level'):]
    latexTableString = latexTableString[:latexTableString.find('Amplification factor')] + '\\multirow{2}{*}{\parbox{2.4cm}{\centering Amplification \\\\ Factor}}' +  latexTableString[latexTableString.find('Amplification factor')+len('Amplification factor'):]
    latexTableString = latexTableString[:latexTableString.find('Packets Per Second')] + '\\multirow{2}{*}{\parbox{3.4cm}{\centering Attacker Packets \\\\ Per Second}}' +  latexTableString[latexTableString.find('Packets Per Second')+len('Packets Per Second'):]
    # Adding a horizontal line for multindex table
    ## Find first \\ after the last column (packets per second)
    columnHeaderEndIndex = latexTableString.find('Second')
    latexTableString = latexTableString[:latexTableString.find('\\\\',columnHeaderEndIndex)+len('\\\\')] + '\cline{2-5}' + latexTableString[latexTableString.find('\\\\',columnHeaderEndIndex)+len('\\\\'):]
    # Add "tab" to label

    # Find caption and label and put at the bottom
    captionIndex = latexTableString.find("\caption")
    labelIndex = latexTableString.find('\label')
    labelStart = latexTableString.find('{',labelIndex)
    # Put 'tab' in label for figure
    latexTableString = latexTableString[:labelStart+1] + 'tab:' + latexTableString[labelStart+1:]
    captionEndIndex = latexTableString.find("}",labelIndex+1)+1
    endTableIndex = latexTableString.find("\end{tabular}")
    endTableEndIndex = endTableIndex + len("\end{tabular}")
    latexTableString = latexTableString[:captionIndex] + latexTableString[captionEndIndex:endTableEndIndex] + latexTableString[captionIndex:captionEndIndex] + latexTableString[endTableEndIndex:]
    return latexTableString

In [831]:
## Get All pcap filenames:
def getFiles(baseDir):
    victimFilenames = []
    attackerFilenames = []
    reflectorFilenames = []
    for root, dirs, files in os.walk(baseDir):
        for file in files:
            if file.endswith(".pcapng"):
                if "victim" in file:
                    victimFilenames.append(os.path.join(root, file))
                if "attacker" in file:
                    attackerFilenames.append(os.path.join(root, file))
                if "reflector" in file:
                    reflectorFilenames.append(os.path.join(root,file))
    return attackerFilenames,victimFilenames,reflectorFilenames

In [832]:
## Get attacker bytes sent
def getAttackStats(attackerFilenames,Protocol,useCachedBytes=False,BytesOrPackets="bytes",Tool="Own"):
    print("Getting Attacker bytes from " + Tool + " tool for " + Protocol + " from pcap files...")
    picklePath = os.path.dirname(os.path.dirname(attackerFilenames[0])) + '/AttackerBytes_' + Protocol + '_' + BytesOrPackets + '_' + Tool + '.pkl'
    if not useCachedBytes:
        pbar = ProgressBar(widgets=widgets)
        attackerStats = []
        for attackFile in pbar(attackerFilenames):
            # Get attack level
            level = int(re.search(r'\d+',attackFile).group(0)) # Find first number in string = level
            attackerDf = pcapToDf(attackFile,True,Tool) 
            if Protocol == "TFTP" or Protocol == "SSDP" or Protocol == "SNMP":
            # By finding the TFTP packets and summing the byte lengths we get the total number of bytes send by the attacker
                attackerDf = attackerDf.loc[attackerDf['Protocol'].isin([Protocol])]
            if BytesOrPackets == "bytes":
                attackerSent = attackerDf["Length"].sum()
            else:
                attackerSent = len(attackerDf)
            pktPerSec = len(attackerDf)/15 # All attacks are 15 sec
            attackerStats.append({'Level':level,'Attacker Outbound':attackerSent,'Packets Per Second':pktPerSec})
        attackerStats = pd.DataFrame(attackerStats).sort_values('Level')
        # Save to pickle file for fast reloading
        attackerStats.to_pickle(picklePath)
    else:
        if os.path.exists(picklePath):
            attackerStats = pd.read_pickle(picklePath)
        else:
            raise Exception("Error no cached pickle file. Run the function with useCachedBytes=False to recalcuate the df, and create the pickle file '" + os.path.basename(picklePath) + "'" )
 
    return attackerStats

In [833]:
## Get Victim bytes received
def getVictimStats(victimFilenames,Protocol,useCachedBytes=False,BytesOrPackets="bytes",Tool="Own"):
    print("Getting Victim bytes from " + Tool + " tool for " + Protocol + " from pcap files...")
    picklePath = os.path.dirname(os.path.dirname(victimFilenames[0])) + '/VictimBytes' + Protocol + '_' + BytesOrPackets + '_' + Tool + '.pkl'
    if not useCachedBytes:
        pbar = ProgressBar(widgets=widgets)
        victimStats = []
        for victimFile in pbar(victimFilenames):
            # Get attack level
            level = int(re.search(r'\d+',victimFile).group(0)) # Find first number in string = level
            victimDf = pcapToDf(victimFile,True,Tool)
            if Protocol == "TFTP":
                # In the victim pcap filtering by destination port 50040 (the tftp servers source port) gives the tftp data transfered to the victim
                victimDf = victimDf.loc[(victimDf['UDP Destination Port'] == 50040)]
            elif Protocol == "SNMP":
                victimDf = victimDf.loc[victimDf['Protocol'].isin([Protocol])]
            elif Protocol == "SSDP":
                victimDf = victimDf.loc[(victimDf['Protocol'] == Protocol) & (victimDf['Destination'] != ipaddress.ip_address('239.255.255.250'))]
            if BytesOrPackets == "bytes":
                victimReceived = victimDf["Length"].sum()
            else:
                victimReceived = len(victimDf)
            victimStats.append({'Level':level,'Victim Inbound':victimReceived}) 
        victimStats = pd.DataFrame(victimStats).sort_values('Level')
        # Save to pickle file for fast reloading
        victimStats.to_pickle(picklePath)
    else:
        if os.path.exists(picklePath):
            victimStats = pd.read_pickle(picklePath)
        else:
            raise Exception("Error no cached pickle file. Run the function with useCachedBytes=False to recalcuate the df, and create the pickle file '" + os.path.basename(picklePath) + "'" )
    return victimStats

In [834]:
## Get Reflector bytes received and sent
def getReflectorStats(reflectorFilenames,Protocol,useCachedBytes=False,BytesOrPackets="bytes",Tool="Own"):
    print("Getting Reflector bytes from " + Tool + " tool for " + Protocol + " from pcap files...")
    picklePath = os.path.dirname(os.path.dirname(reflectorFilenames[0])) + '/ReflectorBytes_' + Protocol + '_' + BytesOrPackets + '_' + Tool + '.pkl'
    if not useCachedBytes:
        pbar = ProgressBar(widgets=widgets)
        reflectorStats = []
        for reflectorFile in pbar(reflectorFilenames):
            # Get attack level
            level = int(re.search(r'\d+',reflectorFile).group(0)) # Find first number in string = level
            reflectorDf = pcapToDf(reflectorFile,True,Tool)
            if Protocol == "TFTP":
                # In the victim pcap filtering by destination port 50040 (the tftp servers source port) gives the tftp data transfered to the victim
                reflectorDfReceived = reflectorDf.loc[(reflectorDf['UDP Source Port'] == 50040)]
                reflectorDfSent = reflectorDf.loc[(reflectorDf['UDP Destination Port'] == 50040)]
            elif Protocol == "SNMP":
                reflectorDfReceived = reflectorDf.loc[(reflectorDf['Protocol'] == Protocol) & (reflectorDf['Info'].str.contains("getBulkRequest"))]
                reflectorDfSent = reflectorDf.loc[(reflectorDf['Protocol'] == Protocol) & (reflectorDf['Info'].str.contains("get-response"))]
            elif Protocol == "SSDP":
                reflectorDfReceived = reflectorDf.loc[(reflectorDf['Protocol'] == Protocol) & (reflectorDf['Destination'] != ipaddress.ip_address('239.255.255.250')) & (reflectorDf["Info"].str.contains("M-SEARCH"))]
                reflectorDfSent = reflectorDf.loc[(reflectorDf['Protocol'] == Protocol) & (reflectorDf['Destination'] != ipaddress.ip_address('239.255.255.250')) & (reflectorDf["Info"].str.contains("HTTP/1.1 200 OK"))]
            if BytesOrPackets == "bytes":
                reflectorReceived = reflectorDfReceived["Length"].sum()
                reflectorSent = reflectorDfSent["Length"].sum()
            else:
                reflectorReceived = len(reflectorDfReceived)
                reflectorSent = len(reflectorDfSent)
            reflectorStats.append({'Level':level,'Reflector Inbound':reflectorReceived,'Reflector Outbound':reflectorSent})
        reflectorStats = pd.DataFrame(reflectorStats).sort_values('Level')
        # Save to pickle file for fast reloading
        reflectorStats.to_pickle(picklePath)
    else:
        if os.path.exists(picklePath):
            reflectorStats = pd.read_pickle(picklePath)
        else:
            raise Exception("Error no cached pickle file. Run the function with useCachedBytes=False to recalcuate the df, and create the pickle file '" + os.path.basename(picklePath) + "'" )
    return reflectorStats

In [835]:
## Data manipulation for presentation
def getStatDf(attackerStats,victimStats,reflectorStats):
    StatDf = pd.DataFrame(columns=pd.MultiIndex.from_tuples([("Victim", "Inbound"), ("Reflector", "Inbound"), ("Reflector", "Outbound"), ("Attacker", "Outbound")]))
    StatDf["Level"] = victimStats['Level']
    StatDf[("Victim","Inbound")] = victimStats['Victim Inbound']
    StatDf[("Reflector","Inbound")] = reflectorStats['Reflector Inbound']
    StatDf[("Reflector","Outbound")] = reflectorStats['Reflector Outbound']
    StatDf[("Attacker","Outbound")] = attackerStats['Attacker Outbound']
    StatDf['Amplification factor'] = StatDf[("Victim","Inbound")] / StatDf[("Attacker","Outbound")]
    StatDf['Packets Per Second'] = attackerStats['Packets Per Second']
    column_to_move = StatDf.pop("Level")
    StatDf.insert(0, "Level", column_to_move)
    return StatDf

In [836]:
## Plotting
def PlotLevelAttackData(attackerStats,victimStats,reflectorStats,Protocol,BytesOrPackets,Tool="own"):
    # Get single index dataframe
    if "Packets Per Second" in attackerStats.columns.to_list():
        attackerStats.pop("Packets Per Second")
    df1 = pd.merge(attackerStats,victimStats,on="Level")
    df2 = pd.merge(df1,reflectorStats,on='Level')
    # Remove 'level' string in level column
    df2["Level"] = df2['Level']
    # Get maximum value in dataframe rounded up to highest exponent (i.e. up to nearest 100M for example)
    maxN = df2.select_dtypes(include=[np.number]).max().max()
    c = 10 ** int(math.log10(maxN)) # Same number of digits as max number
    yaxisRange = [10,math.ceil(maxN/c) * c*2]

    # Plot data
    fig = px.line(df2, 
                title=Protocol + " Test traffic in " + BytesOrPackets,
                x='Level',
                y=['Attacker Outbound','Victim Inbound', 'Reflector Inbound','Reflector Outbound'],
                log_y=True,
                range_y=yaxisRange,
                markers=True,
                color_discrete_map={
                 "Attacker Outbound": "#ED2939",
                 "Victim Inbound": "#007FFF",
                 "Reflector Inbound":"#00A86B",
                 "Reflector Outbound":"orange" 
             })
    fig.update_layout(
        yaxis=dict(
            tickfont=dict(size=16,color="black"),
            title_text=BytesOrPackets.capitalize(),
            titlefont=dict(size=24,color="black")),
        xaxis=dict(
            tickfont=dict(size=16,color="black"),
            title_text="Level",
            titlefont=dict(size=24,color="black")),
        title={'font': {'size': 36,'color':"black"}},
        legend = dict(font = dict(size = 20, color = "black",)),
        legend_title = dict(text="Legend",font = dict(size = 24,color="black"))
        )
    fig.update_traces(line=dict(width=3),marker=dict(size=10))
    fig.show()
    fig.write_image(Protocol + " Test traffic for " + Tool + " tool in " + BytesOrPackets + ".pdf", width=1920/2.4, height=1080/2.4,scale=-2 ,engine='kaleido')

In [837]:
def PlotLevelAttackToolCompareData(df1,df2,Protocol,BytesOrPackets,Tools=["own","lindenhof"]):
    # Get single index dataframe
    if "Packets Per Second" in attackerStats.columns.to_list():
        attackerStats.pop("Packets Per Second")
    dfCombined = pd.merge(df1,df2,on="Level")
    # Remove 'level' string in level column
    dfCombined["Level"] = dfCombined['Level']
    cols = dfCombined.columns.tolist()
    # Reogranize reflector columns to match inbound -> inbound, outbound -> outbound
    if "Reflector" in cols[1]:
        cols = [cols[0],cols[1],cols[3],cols[2],cols[4]]
        dfCombined = dfCombined[cols]

        newnames = {cols[1]:'Reflector Inbound<br>Lindenhof Tool', cols[2]: 'Reflector Outbound<br>Lindenhof Tool',cols[3]:'Reflector Inbound<br>Own Tool',cols[4]:'Reflector Outbound<br>Own Tool'}
        filenameType = cols[1].split()[0]
        
    if "Attacker" in cols[1]:
        newnames = {cols[1]:'Attacker Outbound<br>Lindenhof Tool', cols[2]: 'Attacker Outbound<br>Own Tool',cols[3]:'Victim Inbound<br>Lindenhof Tool',cols[4]:'Victim Inbound<br>Own Tool'}
        filenameType = cols[1].split()[0] + "-" + cols[3].split()[0]
    # Get maximum value in dataframe rounded up to highest exponent (i.e. up to nearest 100M for example)
    maxN = dfCombined.select_dtypes(include=[np.number]).max().max()
    c = 10 ** int(math.log10(maxN)) # Same number of digits as max number
    yaxisRange = [10,math.ceil(maxN/c) * c]
    ynames = dfCombined.columns.to_list()
    ynames.remove("Level")
    # Plot data
    fig = px.line(dfCombined, 
                title=Protocol + " Test traffic in " + BytesOrPackets,
                x='Level',
                y=ynames,
                log_y=True,
                range_y=yaxisRange,
                markers=True,
                color_discrete_map={
                 "Attacker Outbound": "#ED2939",
                 "Victim Inbound": "#007FFF",
                 "Reflector Inbound":"#00A86B",
                 "Reflector Outbound":"orange" 
             })
    fig.update_layout(
        yaxis=dict(
            tickfont=dict(size=16,color="black"),
            title_text=BytesOrPackets.capitalize(),
            titlefont=dict(size=24,color="black")),
        xaxis=dict(
            tickfont=dict(size=16,color="black"),
            title_text="Level",
            titlefont=dict(size=24,color="black")),
        title={'font': {'size': 36,'color':"black"}},
        legend = dict(font = dict(size = 20, color = "black",)),
        legend_title = dict(text="Legend",font = dict(size = 24,color="black"))
        )
    fig.update_traces(line=dict(width=3),marker=dict(size=10))
    fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
    fig.show()
    fig.write_image(Protocol + " " + filenameType + " Test traffic for " + ' and '.join(Tools) + " tool in " + BytesOrPackets + ".pdf", width=1920/2, height=1080/2,scale=-2 ,engine='kaleido')

# Bytes

## TFTP DATA OWN TOOL

In [838]:
# Get pcap files
attackerFilenames,victimFilenames,reflectorFilenames = getFiles('./pcap_tftp_own_tool')
assert len(attackerFilenames) == len(victimFilenames) == len(reflectorFilenames)
Protocol = "TFTP"
useCachedBytes = True
BytesOrPackets="bytes"
Tool="own"

### Reading the data

In [839]:
# Read pcap 
attackerStats = getAttackStats(attackerFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)
victimStats = getVictimStats(victimFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)
reflectorStats = getReflectorStats(reflectorFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)

Getting Attacker bytes from own tool for TFTP from pcap files...
Getting Victim bytes from own tool for TFTP from pcap files...
Getting Reflector bytes from own tool for TFTP from pcap files...


### Displaying the data in latex

In [840]:
#print(tabulate(StatDf,headers='keys',tablefmt='fancy_grid',showindex=False))
StatDf = getStatDf(attackerStats,victimStats,reflectorStats)
latexTableString = StatDf.to_latex(column_format='ccrrrcc',index=False,caption=Protocol + " ARDoS test traffic in " + BytesOrPackets + " for " + Tool + " tool",label=Protocol + "_Test_Traffic_in_" + BytesOrPackets + "_" + Tool,position='H') # For getting the table into the report
latexTableString = refineLatexTableStringScientificPaper(latexTableString)
print(latexTableString)

\begin{table}[H]
\centering

\begin{tabular}{ccrrrcc}
\toprule
\multirow{2}{*}{Level} &  Victim & \multicolumn{2}{c}{Reflector} &  Attacker & \multirow{2}{*}{\parbox{2.4cm}{\centering Amplification \\ Factor}} & \multirow{2}{*}{\parbox{3.4cm}{\centering Attacker Packets \\ Per Second}} \\\cline{2-5}
      & Inbound &   Inbound & Outbound & {Outbound} \\
\midrule
    0 &   51540 &       900 &    51540 &       900 &            57.266667 &           1.000000 \\
    1 &  515400 &      9000 &   515400 &      9000 &            57.266667 &          10.000000 \\
    2 & 1823388 &     87600 &  1823388 &     87600 &            20.814932 &          97.333333 \\
    3 & 2489146 &    715882 &  2489146 &    715800 &             3.477432 &         795.333333 \\
    4 & 3998028 &   4201042 &  3998028 &   4200960 &             0.951694 &        4667.733333 \\
    5 & 3769330 &  10570740 &  3769330 &  10570740 &             0.356581 &       11745.266667 \\
    6 & 3181882 &  41926080 &  3082598 &  42887

### Plotting

In [841]:
PlotLevelAttackData(attackerStats,victimStats,reflectorStats,Protocol,BytesOrPackets,Tool)

## SNMP DATA OWN TOOL

In [842]:
# Get pcap files
attackerFilenames,victimFilenames,reflectorFilenames = getFiles('./pcap_snmp_own_tool')
assert len(attackerFilenames) == len(victimFilenames) == len(reflectorFilenames)
Protocol = "SNMP"
useCachedBytes = True
BytesOrPackets = "bytes"
Tool="own"

### Reading the data

In [843]:
# Read pcap 
attackerStats = getAttackStats(attackerFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)
victimStats = getVictimStats(victimFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)
reflectorStats = getReflectorStats(reflectorFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)

Getting Attacker bytes from own tool for SNMP from pcap files...
Getting Victim bytes from own tool for SNMP from pcap files...
Getting Reflector bytes from own tool for SNMP from pcap files...


### Displaying the data in latex

In [844]:
#print(tabulate(StatDf,headers='keys',tablefmt='fancy_grid',showindex=False))
StatDf = getStatDf(attackerStats,victimStats,reflectorStats)
latexTableString = StatDf.to_latex(column_format='ccrrrcc',index=False,caption=Protocol + " ARDoS test traffic in " + BytesOrPackets + " for " + Tool + " tool",label=Protocol + "_Test_Traffic_in_" + BytesOrPackets + "_" + Tool,position='H') # For getting the table into the report
latexTableString = refineLatexTableStringScientificPaper(latexTableString)
print(latexTableString)

\begin{table}[H]
\centering

\begin{tabular}{ccrrrcc}
\toprule
\multirow{2}{*}{Level} &   Victim & \multicolumn{2}{c}{Reflector} &  Attacker & \multirow{2}{*}{\parbox{2.4cm}{\centering Amplification \\ Factor}} & \multirow{2}{*}{\parbox{3.4cm}{\centering Attacker Packets \\ Per Second}} \\\cline{2-5}
      &  Inbound &   Inbound & Outbound & {Outbound} \\
\midrule
    0 &    21885 &      1140 &    21885 &      1140 &            19.197368 &           1.000000 \\
    1 &   218850 &     11400 &   218850 &     11400 &            19.197368 &          10.000000 \\
    2 &  2103878 &    109592 &  2103878 &    109592 &            19.197368 &          96.133333 \\
    3 & 16054836 &    943388 & 16054836 &    943388 &            17.018275 &         827.533333 \\
    4 & 15679873 &   6193392 & 15679873 &   6193392 &             2.531710 &        5432.800000 \\
    5 & 15285943 &  13757292 & 15285943 &  13757292 &             1.111116 &       12067.800000 \\
    6 & 14941619 &  61406936 & 14425133

### Plotting

In [845]:
PlotLevelAttackData(attackerStats,victimStats,reflectorStats,Protocol,BytesOrPackets,Tool)

## SSDP DATA OWN TOOL

In [846]:
# Get pcap files
attackerFilenames,victimFilenames,reflectorFilenames = getFiles('./pcap_ssdp_own_tool')
assert len(attackerFilenames) == len(victimFilenames) == len(reflectorFilenames)
Protocol = "SSDP"
useCachedBytes = True
BytesOrPackets = "bytes"
Tool="own"

### Reading the data

In [847]:
# Read pcap 
attackerStats = getAttackStats(attackerFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)
victimStats = getVictimStats(victimFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)
reflectorStats = getReflectorStats(reflectorFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)

Getting Attacker bytes from own tool for SSDP from pcap files...
Getting Victim bytes from own tool for SSDP from pcap files...
Getting Reflector bytes from own tool for SSDP from pcap files...


### Displaying the data in latex

In [848]:
#print(tabulate(StatDf,headers='keys',tablefmt='fancy_grid',showindex=False))
StatDf = getStatDf(attackerStats,victimStats,reflectorStats)
latexTableString = StatDf.to_latex(column_format='ccrrrcc',index=False,caption=Protocol + " ARDoS test traffic in " + BytesOrPackets + " for " + Tool + " tool",label=Protocol + "_Test_Traffic_in_" + BytesOrPackets + "_" + Tool,position='H') # For getting the table into the report
latexTableString = refineLatexTableStringScientificPaper(latexTableString)
print(latexTableString)

\begin{table}[H]
\centering

\begin{tabular}{ccrrrcc}
\toprule
\multirow{2}{*}{Level} &    Victim & \multicolumn{2}{c}{Reflector} &  Attacker & \multirow{2}{*}{\parbox{2.4cm}{\centering Amplification \\ Factor}} & \multirow{2}{*}{\parbox{3.4cm}{\centering Attacker Packets \\ Per Second}} \\\cline{2-5}
      &   Inbound &   Inbound & Outbound & {Outbound} \\
\midrule
    0 &      4305 &      2040 &     4305 &      2040 &             2.110294 &           1.000000 \\
    1 &     43050 &     20400 &    43050 &     20400 &             2.110294 &          10.000000 \\
    2 &    419307 &    198696 &   419307 &    198696 &             2.110294 &          97.400000 \\
    3 &   3401237 &   1611736 &  3401237 &   1611736 &             2.110294 &         790.066667 \\
    4 &  21642670 &  10255760 & 21642670 &  10255760 &             2.110294 &        5027.333333 \\
    5 &  50067724 &  22827056 & 48068195 &  23774432 &             2.105948 &       11654.133333 \\
    6 & 101404275 &  80468344 &

### Plotting

In [849]:
PlotLevelAttackData(attackerStats,victimStats,reflectorStats,Protocol,BytesOrPackets,Tool)

# Packets

## TFTP DATA OWN TOOL

In [850]:
# Get pcap files
attackerFilenames,victimFilenames,reflectorFilenames = getFiles('./pcap_tftp_own_tool')
assert len(attackerFilenames) == len(victimFilenames) == len(reflectorFilenames)
Protocol = "TFTP"
useCachedBytes = True
BytesOrPackets="packets"
Tool="own"

### Reading the data

In [851]:
# Read pcap 
attackerStats = getAttackStats(attackerFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)
victimStats = getVictimStats(victimFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)
reflectorStats = getReflectorStats(reflectorFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)

Getting Attacker bytes from own tool for TFTP from pcap files...
Getting Victim bytes from own tool for TFTP from pcap files...
Getting Reflector bytes from own tool for TFTP from pcap files...


### Displaying the data in latex

In [852]:
#print(tabulate(StatDf,headers='keys',tablefmt='fancy_grid',showindex=False))
StatDf = getStatDf(attackerStats,victimStats,reflectorStats)
latexTableString = StatDf.to_latex(column_format='ccrrrcc',index=False,caption=Protocol + " ARDoS test traffic in " + BytesOrPackets + " for " + Tool + " tool",label=Protocol + "_Test_Traffic_in_" + BytesOrPackets + "_" + Tool,position='H') # For getting the table into the report
latexTableString = refineLatexTableStringScientificPaper(latexTableString)
print(latexTableString)

\begin{table}[H]
\centering

\begin{tabular}{ccrrrcc}
\toprule
\multirow{2}{*}{Level} &  Victim & \multicolumn{2}{c}{Reflector} & Attacker & \multirow{2}{*}{\parbox{2.4cm}{\centering Amplification \\ Factor}} & \multirow{2}{*}{\parbox{3.4cm}{\centering Attacker Packets \\ Per Second}} \\\cline{2-5}
      & Inbound &   Inbound & Outbound & {Outbound} \\
\midrule
    0 &     105 &        15 &      105 &       15 &             7.000000 &           1.000000 \\
    1 &    1050 &       150 &     1050 &      150 &             7.000000 &          10.000000 \\
    2 &    4506 &      1460 &     4506 &     1460 &             3.086301 &          97.333333 \\
    3 &   12625 &     11931 &    12625 &    11930 &             1.058256 &         795.333333 \\
    4 &   31026 &     70017 &    31026 &    70016 &             0.443127 &        4667.733333 \\
    5 &   28237 &    176179 &    28237 &   176179 &             0.160274 &       11745.266667 \\
    6 &   21073 &    698768 &    20553 &   714788 &   

### Plotting

In [853]:
PlotLevelAttackData(attackerStats,victimStats,reflectorStats,Protocol,BytesOrPackets,Tool)

## SNMP DATA OWN TOOL

In [854]:
# Get pcap files
attackerFilenames,victimFilenames,reflectorFilenames = getFiles('./pcap_snmp_own_tool')
assert len(attackerFilenames) == len(victimFilenames) == len(reflectorFilenames)
Protocol = "SNMP"
useCachedBytes = True
BytesOrPackets = "packets"
Tool="own"

### Reading the data

In [855]:
# Read pcap 
attackerStats = getAttackStats(attackerFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)
victimStats = getVictimStats(victimFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)
reflectorStats = getReflectorStats(reflectorFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)

Getting Attacker bytes from own tool for SNMP from pcap files...
Getting Victim bytes from own tool for SNMP from pcap files...
Getting Reflector bytes from own tool for SNMP from pcap files...


### Displaying the data in latex

In [856]:
#print(tabulate(StatDf,headers='keys',tablefmt='fancy_grid',showindex=False))
StatDf = getStatDf(attackerStats,victimStats,reflectorStats)
latexTableString = StatDf.to_latex(column_format='ccrrrcc',index=False,caption=Protocol + " ARDoS test traffic in " + BytesOrPackets + " for " + Tool + " tool",label=Protocol + "_Test_Traffic_in_" + BytesOrPackets + "_" + Tool,position='H') # For getting the table into the report
latexTableString = refineLatexTableStringScientificPaper(latexTableString)
print(latexTableString)

\begin{table}[H]
\centering

\begin{tabular}{ccrrrcc}
\toprule
\multirow{2}{*}{Level} &  Victim & \multicolumn{2}{c}{Reflector} & Attacker & \multirow{2}{*}{\parbox{2.4cm}{\centering Amplification \\ Factor}} & \multirow{2}{*}{\parbox{3.4cm}{\centering Attacker Packets \\ Per Second}} \\\cline{2-5}
      & Inbound &   Inbound & Outbound & {Outbound} \\
\midrule
    0 &      15 &        15 &       15 &       15 &             1.000000 &           1.000000 \\
    1 &     150 &       150 &      150 &      150 &             1.000000 &          10.000000 \\
    2 &    1442 &      1442 &     1442 &     1442 &             1.000000 &          96.133333 \\
    3 &   11004 &     12413 &    11004 &    12413 &             0.886490 &         827.533333 \\
    4 &   10747 &     81492 &    10747 &    81492 &             0.131878 &        5432.800000 \\
    5 &   10477 &    181017 &    10477 &   181017 &             0.057879 &       12067.800000 \\
    6 &   10241 &    807986 &     9887 &   839302 &   

### Plotting

In [857]:
PlotLevelAttackData(attackerStats,victimStats,reflectorStats,Protocol,BytesOrPackets,Tool)

## SSDP DATA OWN TOOL

In [858]:
# Get pcap files
attackerFilenames,victimFilenames,reflectorFilenames = getFiles('./pcap_ssdp_own_tool')
assert len(attackerFilenames) == len(victimFilenames) == len(reflectorFilenames)
Protocol = "SSDP"
useCachedBytes = True
BytesOrPackets = "packets"
Tool="own"

### Reading the data

In [859]:
# Read pcap 
attackerStats = getAttackStats(attackerFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)
victimStats = getVictimStats(victimFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)
reflectorStats = getReflectorStats(reflectorFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)

Getting Attacker bytes from own tool for SSDP from pcap files...
Getting Victim bytes from own tool for SSDP from pcap files...
Getting Reflector bytes from own tool for SSDP from pcap files...


### Displaying the data in latex

In [860]:
#print(tabulate(StatDf,headers='keys',tablefmt='fancy_grid',showindex=False))
StatDf = getStatDf(attackerStats,victimStats,reflectorStats)
latexTableString = StatDf.to_latex(column_format='ccrrrcc',index=False,caption=Protocol + " ARDoS test traffic in " + BytesOrPackets + " for " + Tool + " tool",label=Protocol + "_Test_Traffic_in_" + BytesOrPackets + "_" + Tool,position='H') # For getting the table into the report
latexTableString = refineLatexTableStringScientificPaper(latexTableString)
print(latexTableString)

\begin{table}[H]
\centering

\begin{tabular}{ccrrrcc}
\toprule
\multirow{2}{*}{Level} &  Victim & \multicolumn{2}{c}{Reflector} & Attacker & \multirow{2}{*}{\parbox{2.4cm}{\centering Amplification \\ Factor}} & \multirow{2}{*}{\parbox{3.4cm}{\centering Attacker Packets \\ Per Second}} \\\cline{2-5}
      & Inbound &   Inbound & Outbound & {Outbound} \\
\midrule
    0 &      15 &        15 &       15 &       15 &             1.000000 &           1.000000 \\
    1 &     150 &       150 &      150 &      150 &             1.000000 &          10.000000 \\
    2 &    1461 &      1461 &     1461 &     1461 &             1.000000 &          97.400000 \\
    3 &   11851 &     11851 &    11851 &    11851 &             1.000000 &         790.066667 \\
    4 &   75410 &     75410 &    75410 &    75410 &             1.000000 &        5027.333333 \\
    5 &  174452 &    167846 &   167485 &   174812 &             0.997941 &       11654.133333 \\
    6 &  353325 &    591679 &   299372 &   700543 &   

### Plotting

In [861]:
PlotLevelAttackData(attackerStats,victimStats,reflectorStats,Protocol,BytesOrPackets,Tool)

# LindenHoff Tool

## Bytes

### SNMP

In [862]:
# Get pcap files
attackerFilenames,victimFilenames,reflectorFilenames = getFiles('./pcap_snmp_lindenhof_tool')
assert len(attackerFilenames) == len(victimFilenames) == len(reflectorFilenames)
Protocol = "SNMP"
useCachedBytes = True
BytesOrPackets = "bytes"
Tool="lindenhof"

### Reading the data

In [863]:
# Read pcap 
attackerStats = getAttackStats(attackerFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)
victimStats = getVictimStats(victimFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)
reflectorStats = getReflectorStats(reflectorFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)

Getting Attacker bytes from lindenhof tool for SNMP from pcap files...
Getting Victim bytes from lindenhof tool for SNMP from pcap files...
Getting Reflector bytes from lindenhof tool for SNMP from pcap files...


### Displaying the data in latex

In [864]:
#print(tabulate(StatDf,headers='keys',tablefmt='fancy_grid',showindex=False))
StatDf = getStatDf(attackerStats,victimStats,reflectorStats)
latexTableString = StatDf.to_latex(column_format='ccrrrcc',index=False,caption=Protocol + " ARDoS test traffic in " + BytesOrPackets + " for " + Tool + " tool",label=Protocol + "_Test_Traffic_in_" + BytesOrPackets + "_" + Tool,position='H') # For getting the table into the report
latexTableString = refineLatexTableStringScientificPaper(latexTableString)
print(latexTableString)

\begin{table}[H]
\centering

\begin{tabular}{ccrrrcc}
\toprule
\multirow{2}{*}{Level} &   Victim & \multicolumn{2}{c}{Reflector} &  Attacker & \multirow{2}{*}{\parbox{2.4cm}{\centering Amplification \\ Factor}} & \multirow{2}{*}{\parbox{3.4cm}{\centering Attacker Packets \\ Per Second}} \\\cline{2-5}
      &  Inbound &   Inbound & Outbound & {Outbound} \\
\midrule
    0 &    25560 &      1185 &    25560 &      1185 &            21.569620 &           1.000000 \\
    1 &   255600 &     11850 &   255600 &     11850 &            21.569620 &          10.000000 \\
    2 &  1787496 &    118500 &  1787496 &    118500 &            15.084354 &         100.000000 \\
    3 &  1804536 &    800428 &  1804536 &   1185000 &             1.522815 &        1000.000000 \\
    4 &  2126592 &   4971628 &  2126592 &   7643013 &             0.278240 &        6449.800000 \\
    5 &  7069896 &  76827895 &  6936984 & 100544801 &             0.070316 &       84847.933333 \\
    6 & 10184808 & 130319743 &  9488062

### Plotting

In [865]:
PlotLevelAttackData(attackerStats,victimStats,reflectorStats,Protocol,BytesOrPackets,Tool)

## SSDP

In [866]:
# Get pcap files
attackerFilenames,victimFilenames,reflectorFilenames = getFiles('./pcap_ssdp_lindenhof_tool')
assert len(attackerFilenames) == len(victimFilenames) == len(reflectorFilenames)
Protocol = "SSDP"
useCachedBytes = True
BytesOrPackets = "bytes"
Tool="lindenhof"

### Reading the data

In [867]:
# Read pcap 
attackerStats = getAttackStats(attackerFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)
victimStats = getVictimStats(victimFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)
reflectorStats = getReflectorStats(reflectorFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)

Getting Attacker bytes from lindenhof tool for SSDP from pcap files...
Getting Victim bytes from lindenhof tool for SSDP from pcap files...
Getting Reflector bytes from lindenhof tool for SSDP from pcap files...


### Displaying the data in latex

In [868]:
#print(tabulate(StatDf,headers='keys',tablefmt='fancy_grid',showindex=False))
StatDf = getStatDf(attackerStats,victimStats,reflectorStats)
latexTableString = StatDf.to_latex(column_format='ccrrrcc',index=False,caption=Protocol + " ARDoS test traffic in " + BytesOrPackets + " for " + Tool + " tool",label=Protocol + "_Test_Traffic_in_" + BytesOrPackets + "_" + Tool,position='H') # For getting the table into the report
latexTableString = refineLatexTableStringScientificPaper(latexTableString)
print(latexTableString)

\begin{table}[H]
\centering

\begin{tabular}{ccrrrcc}
\toprule
\multirow{2}{*}{Level} &   Victim & \multicolumn{2}{c}{Reflector} &  Attacker & \multirow{2}{*}{\parbox{2.4cm}{\centering Amplification \\ Factor}} & \multirow{2}{*}{\parbox{3.4cm}{\centering Attacker Packets \\ Per Second}} \\\cline{2-5}
      &  Inbound &   Inbound & Outbound & {Outbound} \\
\midrule
    0 &     4290 &      2040 &     4290 &      2040 &             2.102941 &           1.000000 \\
    1 &    42900 &     20400 &    42900 &     20400 &             2.102941 &          10.000000 \\
    2 &   412412 &    204000 &   412412 &    204000 &             2.021627 &         100.000000 \\
    3 &   819962 &   1901688 &  1478048 &   2040000 &             0.401942 &        1000.000000 \\
    4 &  5735444 &   9963904 &  5735444 &  14042680 &             0.408429 &        6883.666667 \\
    5 & 56410640 & 102024072 & 43403074 & 176487200 &             0.319630 &       86513.333333 \\
    6 & 91829738 & 177170872 & 75560628

### Plotting

In [869]:
PlotLevelAttackData(attackerStats,victimStats,reflectorStats,Protocol,BytesOrPackets,Tool)

## Packets

### SNMP

In [870]:
# Get pcap files
attackerFilenames,victimFilenames,reflectorFilenames = getFiles('./pcap_snmp_lindenhof_tool')
assert len(attackerFilenames) == len(victimFilenames) == len(reflectorFilenames)
Protocol = "SNMP"
useCachedBytes = True
BytesOrPackets = "packets"
Tool="lindenhof"

### Reading the data

In [871]:
# Read pcap 
attackerStats = getAttackStats(attackerFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)
victimStats = getVictimStats(victimFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)
reflectorStats = getReflectorStats(reflectorFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)

Getting Attacker bytes from lindenhof tool for SNMP from pcap files...
Getting Victim bytes from lindenhof tool for SNMP from pcap files...
Getting Reflector bytes from lindenhof tool for SNMP from pcap files...


### Displaying the data in latex

In [872]:
#print(tabulate(StatDf,headers='keys',tablefmt='fancy_grid',showindex=False))
StatDf = getStatDf(attackerStats,victimStats,reflectorStats)
latexTableString = StatDf.to_latex(column_format='ccrrrcc',index=False,caption=Protocol + " ARDoS test traffic in " + BytesOrPackets + " for " + Tool + " tool",label=Protocol + "_Test_Traffic_in_" + BytesOrPackets + "_" + Tool,position='H') # For getting the table into the report
latexTableString = refineLatexTableStringScientificPaper(latexTableString)
print(latexTableString)

\begin{table}[H]
\centering

\begin{tabular}{ccrrrcc}
\toprule
\multirow{2}{*}{Level} &  Victim & \multicolumn{2}{c}{Reflector} & Attacker & \multirow{2}{*}{\parbox{2.4cm}{\centering Amplification \\ Factor}} & \multirow{2}{*}{\parbox{3.4cm}{\centering Attacker Packets \\ Per Second}} \\\cline{2-5}
      & Inbound &   Inbound & Outbound & {Outbound} \\
\midrule
    0 &      15 &        15 &       15 &       15 &             1.000000 &           1.000000 \\
    1 &     150 &       150 &      150 &      150 &             1.000000 &          10.000000 \\
    2 &    1049 &      1500 &     1049 &     1500 &             0.699333 &         100.000000 \\
    3 &    1059 &     10132 &     1059 &    15000 &             0.070600 &        1000.000000 \\
    4 &    1248 &     62932 &     1248 &    96747 &             0.012900 &        6449.800000 \\
    5 &    4149 &    972505 &     4071 &  1272719 &             0.003260 &       84847.933333 \\
    6 &    5977 &   1649617 &     5569 &  2271665 &   

### Plotting

In [873]:
PlotLevelAttackData(attackerStats,victimStats,reflectorStats,Protocol,BytesOrPackets,Tool)

## SSDP

In [874]:
# Get pcap files
attackerFilenames,victimFilenames,reflectorFilenames = getFiles('./pcap_ssdp_lindenhof_tool')
assert len(attackerFilenames) == len(victimFilenames) == len(reflectorFilenames)
Protocol = "SSDP"
useCachedBytes = True
BytesOrPackets = "packets"
Tool="lindenhof"

### Reading the data

In [875]:
# Read pcap 
attackerStats = getAttackStats(attackerFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)
victimStats = getVictimStats(victimFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)
reflectorStats = getReflectorStats(reflectorFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)

Getting Attacker bytes from lindenhof tool for SSDP from pcap files...
Getting Victim bytes from lindenhof tool for SSDP from pcap files...
Getting Reflector bytes from lindenhof tool for SSDP from pcap files...


### Displaying the data in latex

In [876]:
#print(tabulate(StatDf,headers='keys',tablefmt='fancy_grid',showindex=False))
StatDf = getStatDf(attackerStats,victimStats,reflectorStats)
latexTableString = StatDf.to_latex(column_format='ccrrrcc',index=False,caption=Protocol + " ARDoS test traffic in " + BytesOrPackets + " for " + Tool + " tool",label=Protocol + "_Test_Traffic_in_" + BytesOrPackets + "_" + Tool,position='H') # For getting the table into the report
latexTableString = refineLatexTableStringScientificPaper(latexTableString)
print(latexTableString)

\begin{table}[H]
\centering

\begin{tabular}{ccrrrcc}
\toprule
\multirow{2}{*}{Level} &  Victim & \multicolumn{2}{c}{Reflector} & Attacker & \multirow{2}{*}{\parbox{2.4cm}{\centering Amplification \\ Factor}} & \multirow{2}{*}{\parbox{3.4cm}{\centering Attacker Packets \\ Per Second}} \\\cline{2-5}
      & Inbound &   Inbound & Outbound & {Outbound} \\
\midrule
    0 &      15 &        15 &       15 &       15 &             1.000000 &           1.000000 \\
    1 &     150 &       150 &      150 &      150 &             1.000000 &          10.000000 \\
    2 &    1442 &      1500 &     1442 &     1500 &             0.961333 &         100.000000 \\
    3 &    2867 &     13983 &     5168 &    15000 &             0.191133 &        1000.000000 \\
    4 &   20054 &     73264 &    20054 &   103255 &             0.194218 &        6883.666667 \\
    5 &  197240 &    750177 &   151759 &  1297700 &             0.151992 &       86513.333333 \\
    6 &  321083 &   1302727 &   264198 &  2141450 &   

### Plotting

In [877]:
PlotLevelAttackData(attackerStats,victimStats,reflectorStats,Protocol,BytesOrPackets,Tool)

## Combining graphs Own tool and Lindenhof tool Bytes

### SNMP

#### Getting filenames Lindenhof

In [878]:
# Get pcap files
attackerFilenames_lindenhof,victimFilenames_lindenhof,reflectorFilenames_lindenhof = getFiles('./pcap_snmp_lindenhof_tool')
assert len(attackerFilenames_lindenhof) == len(victimFilenames_lindenhof) == len(reflectorFilenames_lindenhof)
Protocol = "SNMP"
useCachedBytes = True
BytesOrPackets = "bytes"
Tool="lindenhof"

#### Reading the data Lindenhof

In [879]:
# Read pcap 
attackerStats_lindenhof = getAttackStats(attackerFilenames_lindenhof,Protocol,useCachedBytes,BytesOrPackets,Tool)
victimStats_lindenhof = getVictimStats(victimFilenames_lindenhof,Protocol,useCachedBytes,BytesOrPackets,Tool)
reflectorStats_lindenhof = getReflectorStats(reflectorFilenames_lindenhof,Protocol,useCachedBytes,BytesOrPackets,Tool)

Getting Attacker bytes from lindenhof tool for SNMP from pcap files...
Getting Victim bytes from lindenhof tool for SNMP from pcap files...
Getting Reflector bytes from lindenhof tool for SNMP from pcap files...


#### Getting filenames own tool

In [880]:
# Get pcap files
attackerFilenames_own,victimFilenames_own,reflectorFilenames_own = getFiles('./pcap_snmp_own_tool')
assert len(attackerFilenames_own) == len(victimFilenames_own) == len(reflectorFilenames_own)
Protocol = "SNMP"
useCachedBytes = True
BytesOrPackets = "bytes"
Tool="own"

#### Reading the data

In [881]:
# Read pcap 
attackerStats_own = getAttackStats(attackerFilenames_own,Protocol,useCachedBytes,BytesOrPackets,Tool)
victimStats_own = getVictimStats(victimFilenames_own,Protocol,useCachedBytes,BytesOrPackets,Tool)
reflectorStats_own = getReflectorStats(reflectorFilenames_own,Protocol,useCachedBytes,BytesOrPackets,Tool)

Getting Attacker bytes from own tool for SNMP from pcap files...
Getting Victim bytes from own tool for SNMP from pcap files...
Getting Reflector bytes from own tool for SNMP from pcap files...


#### Combining the data

In [882]:
# Attacker / Victim
if "Packets Per Second" in attackerStats_lindenhof.columns.to_list():
    attackerStats_lindenhof.pop("Packets Per Second")
attackerStats_lindenhof.columns = ['Level','Attacker Outbound Lindenhof Tool']
if len(attackerStats_lindenhof) < len(attackerStats_own):
    attackerStats_lindenhof = attackerStats_lindenhof.append({'Level':10,'Attacker Outbound Lindenhof Tool':None},ignore_index=True)
if "Packets Per Second" in attackerStats_own.columns.to_list():
    attackerStats_own.pop("Packets Per Second")
attackerStats_own.columns = ['Level','Attacker Outbound Own Tool']

victimStats_lindenhof.columns = ['Level','Victim Inbound Lindenhof Tool']
victimStats_own.columns = ['Level','Victim Inbound Own Tool']
if len(victimStats_lindenhof) < len(victimStats_own):
    victimStats_lindenhof = victimStats_lindenhof.append({'Level':10,'Victim Inbound Lindenhof Tool':None},ignore_index=True)

# Reflector
reflectorStats_lindenhof.columns = ['Level','Reflector Inbound Lindenhof Tool','Reflector Outbound Lindenhof Tool']
reflectorStats_own.columns = ['Level','Reflector Inbound Own Tool','Reflector Outbound Own Tool']
if len(reflectorStats_lindenhof) < len(reflectorStats_own):
    reflectorStats_lindenhof = reflectorStats_lindenhof.append({'Level':10,'Reflector Inbound Lindenhof Tool':None,'Reflector Outbound Lindenhof Tool':None},ignore_index=True)

Attack = pd.merge(attackerStats_lindenhof,attackerStats_own,on="Level")
Victim = pd.merge(victimStats_lindenhof,victimStats_own,on="Level")
Tools=["own","lindenhof"]

#### Plotting Attack/victim

In [883]:
PlotLevelAttackToolCompareData(Attack,Victim,Protocol,BytesOrPackets,Tools)

#### Plotting Reflector

In [884]:
PlotLevelAttackToolCompareData(reflectorStats_lindenhof,reflectorStats_own,Protocol,BytesOrPackets,Tools)

### SSDP

#### Getting filenames Lindenhof

In [885]:
# Get pcap files
attackerFilenames_lindenhof,victimFilenames_lindenhof,reflectorFilenames_lindenhof = getFiles('./pcap_ssdp_lindenhof_tool')
assert len(attackerFilenames_lindenhof) == len(victimFilenames_lindenhof) == len(reflectorFilenames_lindenhof)
Protocol = "SSDP"
useCachedBytes = True
BytesOrPackets = "bytes"
Tool="lindenhof"

#### Reading the data Lindenhof

In [886]:
# Read pcap 
attackerStats_lindenhof = getAttackStats(attackerFilenames_lindenhof,Protocol,useCachedBytes,BytesOrPackets,Tool)
victimStats_lindenhof = getVictimStats(victimFilenames_lindenhof,Protocol,useCachedBytes,BytesOrPackets,Tool)
reflectorStats_lindenhof = getReflectorStats(reflectorFilenames_lindenhof,Protocol,useCachedBytes,BytesOrPackets,Tool)

Getting Attacker bytes from lindenhof tool for SSDP from pcap files...
Getting Victim bytes from lindenhof tool for SSDP from pcap files...
Getting Reflector bytes from lindenhof tool for SSDP from pcap files...


#### Getting filenames own tool

In [887]:
# Get pcap files
attackerFilenames_own,victimFilenames_own,reflectorFilenames_own = getFiles('./pcap_ssdp_own_tool')
assert len(attackerFilenames_own) == len(victimFilenames_own) == len(reflectorFilenames_own)
Protocol = "SSDP"
useCachedBytes = True
BytesOrPackets = "bytes"
Tool="own"

#### Reading the data

In [888]:
# Read pcap 
attackerStats_own = getAttackStats(attackerFilenames_own,Protocol,useCachedBytes,BytesOrPackets,Tool)
victimStats_own = getVictimStats(victimFilenames_own,Protocol,useCachedBytes,BytesOrPackets,Tool)
reflectorStats_own = getReflectorStats(reflectorFilenames_own,Protocol,useCachedBytes,BytesOrPackets,Tool)

Getting Attacker bytes from own tool for SSDP from pcap files...
Getting Victim bytes from own tool for SSDP from pcap files...
Getting Reflector bytes from own tool for SSDP from pcap files...


#### Combining the data

In [889]:
# Attacker / Victim
if "Packets Per Second" in attackerStats_lindenhof.columns.to_list():
    attackerStats_lindenhof.pop("Packets Per Second")
attackerStats_lindenhof.columns = ['Level','Attacker Outbound Lindenhof Tool']
if len(attackerStats_lindenhof) < len(attackerStats_own):
    attackerStats_lindenhof = attackerStats_lindenhof.append({'Level':10,'Attacker Outbound Lindenhof Tool':None},ignore_index=True)
if "Packets Per Second" in attackerStats_own.columns.to_list():
    attackerStats_own.pop("Packets Per Second")
attackerStats_own.columns = ['Level','Attacker Outbound Own Tool']

victimStats_lindenhof.columns = ['Level','Victim Inbound Lindenhof Tool']
victimStats_own.columns = ['Level','Victim Inbound Own Tool']
if len(victimStats_lindenhof) < len(victimStats_own):
    victimStats_lindenhof = victimStats_lindenhof.append({'Level':10,'Victim Inbound Lindenhof Tool':None},ignore_index=True)

# Reflector
reflectorStats_lindenhof.columns = ['Level','Reflector Inbound Lindenhof Tool','Reflector Outbound Lindenhof Tool']
reflectorStats_own.columns = ['Level','Reflector Inbound Own Tool','Reflector Outbound Own Tool']
if len(reflectorStats_lindenhof) < len(reflectorStats_own):
    reflectorStats_lindenhof = reflectorStats_lindenhof.append({'Level':10,'Reflector Inbound Lindenhof Tool':None,'Reflector Outbound Lindenhof Tool':None},ignore_index=True)


Attack = pd.merge(attackerStats_lindenhof,attackerStats_own,on="Level")
Victim = pd.merge(victimStats_lindenhof,victimStats_own,on="Level")
Tools=["own","lindenhof"]

#### Plotting

In [890]:
PlotLevelAttackToolCompareData(Attack,Victim,Protocol,BytesOrPackets,Tools)

In [891]:
PlotLevelAttackToolCompareData(reflectorStats_lindenhof,reflectorStats_own,Protocol,BytesOrPackets,Tools)

## Combining graphs Own tool and Lindenhof tool Types Packets

### SNMP

#### Getting filenames Lindenhof

In [892]:
# Get pcap files
attackerFilenames_lindenhof,victimFilenames_lindenhof,reflectorFilenames_lindenhof = getFiles('./pcap_snmp_lindenhof_tool')
assert len(attackerFilenames_lindenhof) == len(victimFilenames_lindenhof) == len(reflectorFilenames_lindenhof)
Protocol = "SNMP"
useCachedBytes = True
BytesOrPackets = "packets"
Tool="lindenhof"

#### Reading the data Lindenhof

In [893]:
# Read pcap 
attackerStats_lindenhof = getAttackStats(attackerFilenames_lindenhof,Protocol,useCachedBytes,BytesOrPackets,Tool)
victimStats_lindenhof = getVictimStats(victimFilenames_lindenhof,Protocol,useCachedBytes,BytesOrPackets,Tool)
reflectorStats_lindenhof = getReflectorStats(reflectorFilenames_lindenhof,Protocol,useCachedBytes,BytesOrPackets,Tool)

Getting Attacker bytes from lindenhof tool for SNMP from pcap files...
Getting Victim bytes from lindenhof tool for SNMP from pcap files...
Getting Reflector bytes from lindenhof tool for SNMP from pcap files...


#### Getting filenames own tool

In [894]:
# Get pcap files
attackerFilenames_own,victimFilenames_own,reflectorFilenames_own = getFiles('./pcap_snmp_own_tool')
assert len(attackerFilenames_own) == len(victimFilenames_own) == len(reflectorFilenames_own)
Protocol = "SNMP"
useCachedBytes = True
BytesOrPackets = "packets"
Tool="own"

In [895]:
# Read pcap 
attackerStats_own = getAttackStats(attackerFilenames_own,Protocol,useCachedBytes,BytesOrPackets,Tool)
victimStats_own = getVictimStats(victimFilenames_own,Protocol,useCachedBytes,BytesOrPackets,Tool)
reflectorStats_own = getReflectorStats(reflectorFilenames_own,Protocol,useCachedBytes,BytesOrPackets,Tool)

Getting Attacker bytes from own tool for SNMP from pcap files...
Getting Victim bytes from own tool for SNMP from pcap files...
Getting Reflector bytes from own tool for SNMP from pcap files...


#### Combining the data

In [896]:
# Attacker / Victim
if "Packets Per Second" in attackerStats_lindenhof.columns.to_list():
    attackerStats_lindenhof.pop("Packets Per Second")
attackerStats_lindenhof.columns = ['Level','Attacker Outbound Lindenhof Tool']
if len(attackerStats_lindenhof) < len(attackerStats_own):
    attackerStats_lindenhof = attackerStats_lindenhof.append({'Level':10,'Attacker Outbound Lindenhof Tool':None},ignore_index=True)
if "Packets Per Second" in attackerStats_own.columns.to_list():
    attackerStats_own.pop("Packets Per Second")
attackerStats_own.columns = ['Level','Attacker Outbound Own Tool']

victimStats_lindenhof.columns = ['Level','Victim Inbound Lindenhof Tool']
victimStats_own.columns = ['Level','Victim Inbound Own Tool']
if len(victimStats_lindenhof) < len(victimStats_own):
    victimStats_lindenhof = victimStats_lindenhof.append({'Level':10,'Victim Inbound Lindenhof Tool':None},ignore_index=True)

# Reflector
reflectorStats_lindenhof.columns = ['Level','Reflector Inbound Lindenhof Tool','Reflector Outbound Lindenhof Tool']
reflectorStats_own.columns = ['Level','Reflector Inbound Own Tool','Reflector Outbound Own Tool']
if len(reflectorStats_lindenhof) < len(reflectorStats_own):
    reflectorStats_lindenhof = reflectorStats_lindenhof.append({'Level':10,'Reflector Inbound Lindenhof Tool':None,'Reflector Outbound Lindenhof Tool':None},ignore_index=True)

Attack = pd.merge(attackerStats_lindenhof,attackerStats_own,on="Level")
Victim = pd.merge(victimStats_lindenhof,victimStats_own,on="Level")
Tools=["own","lindenhof"]

#### Plotting

In [900]:
# Axis are weird for packet plots so we have to redefine the function
def PlotLevelAttackToolCompareData(df1,df2,Protocol,BytesOrPackets,Tools=["own","lindenhof"]):
    # Get single index dataframe
    if "Packets Per Second" in attackerStats.columns.to_list():
        attackerStats.pop("Packets Per Second")
    dfCombined = pd.merge(df1,df2,on="Level")
    # Remove 'level' string in level column
    dfCombined["Level"] = dfCombined['Level']
    cols = dfCombined.columns.tolist()
    # Reogranize reflector columns to match inbound -> inbound, outbound -> outbound
    if "Reflector" in cols[1]:
        cols = [cols[0],cols[1],cols[3],cols[2],cols[4]]
        dfCombined = dfCombined[cols]

        newnames = {cols[1]:'Reflector Inbound<br>Lindenhof Tool', cols[2]: 'Reflector Outbound<br>Lindenhof Tool',cols[3]:'Reflector Inbound<br>Own Tool',cols[4]:'Reflector Outbound<br>Own Tool'}
        filenameType = cols[1].split()[0]
        
    if "Attacker" in cols[1]:
        newnames = {cols[1]:'Attacker Outbound<br>Lindenhof Tool', cols[2]: 'Attacker Outbound<br>Own Tool',cols[3]:'Victim Inbound<br>Lindenhof Tool',cols[4]:'Victim Inbound<br>Own Tool'}
        filenameType = cols[1].split()[0] + "-" + cols[3].split()[0]
    # Get maximum value in dataframe rounded up to highest exponent (i.e. up to nearest 100M for example)
    maxN = dfCombined.select_dtypes(include=[np.number]).max().max()
    c = 10 ** int(math.log10(maxN)) # Same number of digits as max number
    yaxisRange = [10,math.ceil(maxN/c) * c*2]
    ynames = dfCombined.columns.to_list()
    ynames.remove("Level")
    # Plot data
    fig = px.line(dfCombined, 
                title=Protocol + " Test traffic in " + BytesOrPackets,
                x='Level',
                y=ynames,
                log_y=True,
                range_y=yaxisRange,
                markers=True,
                color_discrete_map={
                 "Attacker Outbound": "#ED2939",
                 "Victim Inbound": "#007FFF",
                 "Reflector Inbound":"#00A86B",
                 "Reflector Outbound":"orange" 
             })
    fig.update_layout(
        yaxis=dict(
            tickfont=dict(size=16,color="black"),
            title_text=BytesOrPackets.capitalize(),
            titlefont=dict(size=24,color="black"),
            tickmode = 'array',
            tickvals = [10, 100, 1000, 10000, 100000, 1000000],
            ticktext = ['10','100','1000','10k','100k','1M'])
        ,
        xaxis=dict(
            tickfont=dict(size=16,color="black"),
            title_text="Level",
            titlefont=dict(size=24,color="black")),
        title={'font': {'size': 36,'color':"black"}},
        legend = dict(font = dict(size = 20, color = "black",)),
        legend_title = dict(text="Legend",font = dict(size = 24,color="black"))
        )
    fig.update_traces(line=dict(width=3),marker=dict(size=10))
    fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
    fig.show()
    fig.write_image(Protocol + " " + filenameType + " Test traffic for " + ' and '.join(Tools) + " tool in " + BytesOrPackets + ".pdf", width=1920/2, height=1080/2,scale=-2 ,engine='kaleido')

In [901]:
PlotLevelAttackToolCompareData(Attack,Victim,Protocol,BytesOrPackets,Tools)

In [902]:
PlotLevelAttackToolCompareData(reflectorStats_lindenhof,reflectorStats_own,Protocol,BytesOrPackets,Tools)

### SSDP

#### Getting filenames Lindenhof

In [903]:
# Get pcap files
attackerFilenames_lindenhof,victimFilenames_lindenhof,reflectorFilenames_lindenhof = getFiles('./pcap_ssdp_lindenhof_tool')
assert len(attackerFilenames_lindenhof) == len(victimFilenames_lindenhof) == len(reflectorFilenames_lindenhof)
Protocol = "SSDP"
useCachedBytes = True
BytesOrPackets = "packets"
Tool="lindenhof"

#### Reading the data Lindenhof

In [904]:
# Read pcap 
attackerStats_lindenhof = getAttackStats(attackerFilenames_lindenhof,Protocol,useCachedBytes,BytesOrPackets,Tool)
victimStats_lindenhof = getVictimStats(victimFilenames_lindenhof,Protocol,useCachedBytes,BytesOrPackets,Tool)
reflectorStats_lindenhof = getReflectorStats(reflectorFilenames_lindenhof,Protocol,useCachedBytes,BytesOrPackets,Tool)

Getting Attacker bytes from lindenhof tool for SSDP from pcap files...
Getting Victim bytes from lindenhof tool for SSDP from pcap files...
Getting Reflector bytes from lindenhof tool for SSDP from pcap files...


#### Getting filenames own tool

In [905]:
# Get pcap files
attackerFilenames_own,victimFilenames_own,reflectorFilenames_own = getFiles('./pcap_ssdp_own_tool')
assert len(attackerFilenames_own) == len(victimFilenames_own) == len(reflectorFilenames_own)
Protocol = "SSDP"
useCachedBytes = True
BytesOrPackets = "packets"
Tool="own"

In [906]:
# Read pcap 
attackerStats_own = getAttackStats(attackerFilenames_own,Protocol,useCachedBytes,BytesOrPackets,Tool)
victimStats_own = getVictimStats(victimFilenames_own,Protocol,useCachedBytes,BytesOrPackets,Tool)
reflectorStats_own = getReflectorStats(reflectorFilenames_own,Protocol,useCachedBytes,BytesOrPackets,Tool)

Getting Attacker bytes from own tool for SSDP from pcap files...
Getting Victim bytes from own tool for SSDP from pcap files...
Getting Reflector bytes from own tool for SSDP from pcap files...


#### Combining the data

In [907]:
# Attacker / Victim
if "Packets Per Second" in attackerStats_lindenhof.columns.to_list():
    attackerStats_lindenhof.pop("Packets Per Second")
attackerStats_lindenhof.columns = ['Level','Attacker Outbound Lindenhof Tool']
if len(attackerStats_lindenhof) < len(attackerStats_own):
    attackerStats_lindenhof = attackerStats_lindenhof.append({'Level':10,'Attacker Outbound Lindenhof Tool':None},ignore_index=True)
if "Packets Per Second" in attackerStats_own.columns.to_list():
    attackerStats_own.pop("Packets Per Second")
attackerStats_own.columns = ['Level','Attacker Outbound Own Tool']

victimStats_lindenhof.columns = ['Level','Victim Inbound Lindenhof Tool']
victimStats_own.columns = ['Level','Victim Inbound Own Tool']
if len(victimStats_lindenhof) < len(victimStats_own):
    victimStats_lindenhof = victimStats_lindenhof.append({'Level':10,'Victim Inbound Lindenhof Tool':None},ignore_index=True)

# Reflector
reflectorStats_lindenhof.columns = ['Level','Reflector Inbound Lindenhof Tool','Reflector Outbound Lindenhof Tool']
reflectorStats_own.columns = ['Level','Reflector Inbound Own Tool','Reflector Outbound Own Tool']
if len(reflectorStats_lindenhof) < len(reflectorStats_own):
    reflectorStats_lindenhof = reflectorStats_lindenhof.append({'Level':10,'Reflector Inbound Lindenhof Tool':None,'Reflector Outbound Lindenhof Tool':None},ignore_index=True)

Attack = pd.merge(attackerStats_lindenhof,attackerStats_own,on="Level")
Victim = pd.merge(victimStats_lindenhof,victimStats_own,on="Level")
Tools=["own","lindenhof"]

#### Plotting

In [911]:
# Axis are weird for packet plots so we have to redefine the function
def PlotLevelAttackToolCompareData(df1,df2,Protocol,BytesOrPackets,Tools=["own","lindenhof"]):
    # Get single index dataframe
    if "Packets Per Second" in attackerStats.columns.to_list():
        attackerStats.pop("Packets Per Second")
    dfCombined = pd.merge(df1,df2,on="Level")
    # Remove 'level' string in level column
    dfCombined["Level"] = dfCombined['Level']
    cols = dfCombined.columns.tolist()
    # Reogranize reflector columns to match inbound -> inbound, outbound -> outbound
    if "Reflector" in cols[1]:
        cols = [cols[0],cols[1],cols[3],cols[2],cols[4]]
        dfCombined = dfCombined[cols]

        newnames = {cols[1]:'Reflector Inbound<br>Lindenhof Tool', cols[2]: 'Reflector Outbound<br>Lindenhof Tool',cols[3]:'Reflector Inbound<br>Own Tool',cols[4]:'Reflector Outbound<br>Own Tool'}
        filenameType = cols[1].split()[0]
        
    if "Attacker" in cols[1]:
        newnames = {cols[1]:'Attacker Outbound<br>Lindenhof Tool', cols[2]: 'Attacker Outbound<br>Own Tool',cols[3]:'Victim Inbound<br>Lindenhof Tool',cols[4]:'Victim Inbound<br>Own Tool'}
        filenameType = cols[1].split()[0] + "-" + cols[3].split()[0]
    # Get maximum value in dataframe rounded up to highest exponent (i.e. up to nearest 100M for example)
    maxN = dfCombined.select_dtypes(include=[np.number]).max().max()
    c = 10 ** int(math.log10(maxN)) # Same number of digits as max number
    yaxisRange = [10,math.ceil(maxN/c) * c*2]
    ynames = dfCombined.columns.to_list()
    ynames.remove("Level")
    # Plot data
    fig = px.line(dfCombined, 
                title=Protocol + " Test traffic in " + BytesOrPackets,
                x='Level',
                y=ynames,
                log_y=True,
                range_y=yaxisRange,
                markers=True,
                color_discrete_map={
                 "Attacker Outbound": "#ED2939",
                 "Victim Inbound": "#007FFF",
                 "Reflector Inbound":"#00A86B",
                 "Reflector Outbound":"orange" 
             })
    fig.update_layout(
        yaxis=dict(
            tickfont=dict(size=16,color="black"),
            title_text=BytesOrPackets.capitalize(),
            titlefont=dict(size=24,color="black"),
            tickmode = 'array',
            tickvals = [10, 100, 1000, 10000, 100000, 1000000],
            ticktext = ['10','100','1000','10k','100k','1M'])
        ,
        xaxis=dict(
            tickfont=dict(size=16,color="black"),
            title_text="Level",
            titlefont=dict(size=24,color="black")),
        title={'font': {'size': 36,'color':"black"}},
        legend = dict(font = dict(size = 20, color = "black",)),
        legend_title = dict(text="Legend",font = dict(size = 24,color="black"))
        )
    fig.update_traces(line=dict(width=3),marker=dict(size=10))
    fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
    fig.show()
    fig.write_image(Protocol + " " + filenameType + " Test traffic for " + ' and '.join(Tools) + " tool in " + BytesOrPackets + ".pdf", width=1920/2, height=1080/2,scale=-2 ,engine='kaleido')

In [912]:
PlotLevelAttackToolCompareData(Attack,Victim,Protocol,BytesOrPackets,Tools)

In [913]:
PlotLevelAttackToolCompareData(reflectorStats_lindenhof,reflectorStats_own,Protocol,BytesOrPackets,Tools)